In [1]:
import nltk
import pandas as pd
import pickle
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from tensorflow import keras

/home/parth/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-09 20:38:15.066433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 20:38:16.200980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2023-02-09 20:38:16.201084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load d

In [1]:
import pandas as pd
df = pd.read_json("smalldataset/newsdataset.json", lines=True)

In [ ]:
model = spacy.load("en_core_web_trf")
df.head(5)

In [ ]:
df.drop(columns=["authors","link","date"], inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df["fullnews"] = df["headline"] +" "+ df["short_description"]

In [ ]:
df = df[["category","fullnews"]]

In [ ]:
df.sample(5)

In [ ]:
'''dropcat = ["RELIGION", "LATINO VOICES", "FIFTY", "WEIRD NEWS", "ENVIRONMENT", "QUEER VOICES", "GOOD NEWS", "MEDIA", "WEDDINGS", "GREEN", "TASTE", "WORLD NEWS", "IMPACT", "WOMEN"]

for i in dropcat:
    df.drop(df.loc[df["category"]==i].index, inplace=True, axis=0)

df.loc[df["category"]=="COLLEGE"] = ["EDUCATION"]
df.loc[df["category"]=="ARTS & CULTURE"] = ["CULTURE & ARTS"]
df.loc[df["category"]=="ARTS"] = ["CULTURE & ARTS"]
df.loc[(df["category"]=="SCIENCE") | (df["category"]=="TECH")] = ["SCIENCE & TECH"]
df.loc[df["category"]=="WORLDPOST"] = ["THE WORLDPOST"]
df.loc[df["category"]=="STYLE"] = ["STYLE & BEAUTY"]
df.loc[df["category"]=="MONEY"] = ["BUSINESS"]

df.reset_index(inplace=True, drop=True)
df.to_json("smalldataset/newsdataset.json", lines=True, orient="records")'''

In [ ]:
not_allowed_word_types = ["PUNCT", "DET", "PART", "CCONJ", "SPACE"]
token_news = []
count = 0

for i in df["fullnews"]:
    tokenised = model(i)
    smalltoken = [j.lemma_ for j in 
                  tokenised if j.pos_ not in not_allowed_word_types]
    token_news.append(smalltoken)
    count += 1
    if count%5000==0:
        print(count)

In [ ]:
df["tokenizednews"] = token_news

In [36]:
stop_words = stopwords.words("english")

In [37]:
all_words = [w.lower()
            for p in df["tokenizednews"]
            for w in p
            if w.lower() not in stop_words]

In [ ]:
"""allowed_word_types = ["J"]


for p in df["tokenizednews"]:
    pos = nltk.pos_tag(p)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())"""

In [ ]:
"""print(len(all_words))
all_words = nltk.FreqDist(all_words)
print(len(all_words))"""

In [38]:
print(len(all_words))
all_wordsfre = nltk.FreqDist(all_words)
print(len(all_wordsfre))

2918465
69704


In [ ]:
all_wordsfre.most_common(15)

In [39]:
word_features = [w[0] for w in all_wordsfre.most_common()]

In [ ]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features

In [ ]:
df.shape[0]

In [ ]:
'''df.to_json("smalldataset/newsdatasetjson", lines=True, orient="records")'''

In [ ]:
file = open("smalldataset/word_features.pickle","wb")
pickle.dump(word_features, file)

In [ ]:
file = open("smalldataset/word_features.pickle", "rb")
word_features = pickle.load(file)
word_features = word_features[:10000]

In [ ]:
countvector = CountVectorizer().fit(word_features)
not_words = [i for i in word_features if i not in countvector.vocabulary_]

for i in not_words:
    word_features.remove(i)

word_features = word_features[:8000]
countvector = CountVectorizer().fit(word_features)

In [ ]:
encoder = OneHotEncoder()
encoder.fit(np.array(df["category"]).reshape(-1,1))

In [ ]:
df = pd.read_json("smalldataset/newsdataset.json", lines=True, chunksize=5000)

In [ ]:
def find_features(document):
    features = countvector.transform(document).toarray()
            
    return features

In [ ]:

def get_features():
    for data in df:

        Y = encoder.transform(np.array(data["category"]).reshape(-1,1)).toarray()

        X = find_features(data["fullnews"])

        break
        
    return X, Y
        

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import relu

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
from keras.layers import Bidirectional, Embedding, Conv1D, MaxPooling1D, SimpleRNN, LSTM, GRU, Dropout, Dense, Input

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
model = keras.Sequential([
    keras.layers.Input(shape=8000),
    keras.layers.Embedding(8000, 3),
    keras.layers.Flatten(),
    keras.layers.Dense(1000, activation="LeakyReLU"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(800, activation="LeakyReLU"),
    keras.layers.Dense(400, activation="LeakyReLU"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(200, activation="LeakyReLU"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(20, activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model2 = keras.Sequential()
model2.add(Input(shape=8000))
model2.add(Embedding(8000, 3))
model2.add(Bidirectional(LSTM(64, dropout=0.1, recurrent_dropout=0.10, activation='tanh', return_sequences=True)))
model2.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model2.add(Bidirectional(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model2.add(Conv1D(72, 3, activation='relu'))
model2.add(MaxPooling1D(2))
model2.add(SimpleRNN(64, activation='tanh', dropout=0.2, recurrent_dropout=0.20, return_sequences=True))
model2.add(GRU(64, recurrent_dropout=0.20, recurrent_regularizer='l1_l2'))
model2.add(Dropout(0.2))
model2.add(Dense(20, activation='softmax'))
model2.summary()



model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [6]:
labencoder = LabelEncoder()
#encoder.classes_ = np.load("labelencoder15.npy", allow_pickle=True)

In [7]:
labencoder.fit(df["category"])
#np.save("smalldataset/labelencoder.npy", labencoder.classes_)

LabelEncoder()

In [ ]:
len(encoder.classes_)

In [ ]:
X_test = []
Y_test = []
checkpoint = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)

In [ ]:
for i in range(10):
    X, Y = get_features()
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    X_test.append(x_test)
    Y_test.append(y_test)
    print(X.shape, Y.shape)
    
    
    model2.fit(x_train, y_train, validation_split=0.2, epochs=5, batch_size=5, callbacks=checkpoint)
    print(f"Accuracy after {i} traindata")
    model2.evaluate(x_test,y_test)
    #model = keras.models.load_model("model.h5")
    

In [ ]:
model = keras.models.load_model("model.h5")

In [ ]:
X, Y = get_features()

In [ ]:
Y_pred = model.predict(X)

In [ ]:
y_pred_arg = np.argmax(Y_pred, axis = 1)

In [ ]:
max(Y_pred[0])

In [ ]:
labencoder.classes_[y_pred_arg[:5]]

In [ ]:
Y[:5]

In [ ]:
encoder.categories_

In [ ]:
labencoder.classes_

In [ ]:
kk = np.array(X_test)

In [ ]:
kk.shape

In [ ]:
kk = kk.reshape(-1, 8000)

In [ ]:
yy = np.array(Y_test)

In [ ]:
yy = yy.reshape(-1, 20)

In [ ]:
model.evaluate(kk, yy, batch_size=5)

In [ ]:
model.summary()

In [ ]:
model.summary()

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# model building imports
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Conv1D, SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1D, LSTM, GRU, Input
from keras.models import Sequential
from keras.regularizers import L1L2
#from wordcloud import WordCloud, STOPWORDS

In [4]:
df

,category,fullnews,tokenizednews,reducedtoken
0,CRIME,there were 2 mass shootings in texas last week...,"[there, be, 2, mass, shooting, in, texas, last...","[mass, shooting, texas, last, week, tv, leave,..."
1,ENTERTAINMENT,will smith joins diplo and nicky jam for the 2...,"[will, smith, join, diplo, nicky, jam, for, 20...","[smith, join, diplo, nicky, jam, 2018, world, ..."
2,ENTERTAINMENT,hugh grant marries for the first time at age 5...,"[hugh, grant, marry, for, first, time, at, age...","[hugh, grant, marry, first, time, age, 57, act..."
3,ENTERTAINMENT,jim carrey blasts 'castrato' adam schiff and d...,"[jim, carrey, blast, castrato, adam, schiff, d...","[jim, carrey, blast, castrato, adam, schiff, d..."
4,ENTERTAINMENT,julianna margulies uses donald trump poop bags...,"[julianna, margulies, use, donald, trump, poop...","[julianna, margulies, use, donald, trump, poop..."
...,...,...,...,...
163747,SCIENCE & TECH,rim ceo thorsten heins' 'significant' plans fo...,"[rim, ceo, thorsten, heins, significant, plan,...","[rim, ceo, thorsten, heins, significant, plan,..."
163748,SPORTS,maria sharapova stunned by victoria azarenka i...,"[maria, sharapova, stun, by, victoria, azarenk...","[maria, sharapova, stun, victoria, azarenka, a..."
163749,SPORTS,"giants over patriots, jets over colts among m...","[giant, over, patriot, jet, over, colt, among,...","[giant, patriot, jet, colt, among, improbable,..."
163750,SPORTS,aldon smith arrested: 49ers linebacker busted ...,"[aldon, smith, arrest, 49ers, linebacker, bust...","[aldon, smith, arrest, 49ers, linebacker, bust..."


In [ ]:
stop_words = stopwords.words("english")

In [ ]:
all_words = [w.lower()
            for p in df["tokenizednews"]
            for w in p
            if w.lower() not in stop_words]

In [ ]:
print(len(all_words))
all_wordsfre = nltk.FreqDist(all_words)
print(len(all_wordsfre))

2918465
69704


In [ ]:
word_features = [w[0] for w in all_wordsfre.most_common()]
countvector = CountVectorizer(stop_words=stop_words).fit(word_features)

In [ ]:
vocab = list(countvector.vocabulary_.keys())

In [ ]:
numwords = len(vocab)

In [ ]:
X = df["reducedtoken"]
tokenizer = Tokenizer(num_words=numwords, oov_token='<00V>') 
tokenizer.fit_on_texts(X)
train_seq = tokenizer.texts_to_sequences(X)

In [71]:
templist = []
for i in df["tokenizednews"]:
    j = [k for k in i if k in vocab]
    templist.append(j)

In [ ]:
train_padseq = pad_sequences(train_seq, maxlen=136)

In [ ]:
encoder = OneHotEncoder()
encoder.fit(np.array(df["category"]).reshape(-1,1))

OneHotEncoder()

In [ ]:
Y = encoder.transform(np.array(df["category"]).reshape(-1,1)).toarray()

In [23]:
checkpoint = keras.callbacks.ModelCheckpoint("model5.h5", save_best_only=True)
model = keras.Sequential([
    keras.layers.Input(shape=136),
    keras.layers.Embedding(8000, 180),
    keras.layers.GRU(400, return_sequences=True),
    keras.layers.GRU(200, return_sequences=True),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(800, activation="LeakyReLU"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="LeakyReLU"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(300, activation="LeakyReLU"),
    keras.layers.Dense(200, activation="LeakyReLU"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(20, activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 136, 180)          1440000   
                                                                 
 gru_2 (GRU)                 (None, 136, 400)          698400    
                                                                 
 gru_3 (GRU)                 (None, 136, 200)          361200    
                                                                 
 flatten_1 (Flatten)         (None, 27200)             0         
                                                                 
 batch_normalization_2 (Batc  (None, 27200)            108800    
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 27200)             0         
                                                      

In [24]:
padseq_train, padseq_test, y_train, y_test = train_test_split(train_padseq, Y, test_size=0.2)

In [27]:
model.fit(padseq_train, y_train, epochs=5, validation_split=0.2, callbacks=checkpoint, batch_size=20)

Epoch 1/5
5240/5240 [==============================] - 267s 51ms/step - loss: 0.6917 - accuracy: 0.7914 - val_loss: 1.7026 - val_accuracy: 0.6057
Epoch 2/5
5240/5240 [==============================] - 275s 52ms/step - loss: 0.5943 - accuracy: 0.8187 - val_loss: 4.8275 - val_accuracy: 0.4074
Epoch 3/5
5240/5240 [==============================] - 273s 52ms/step - loss: 0.5096 - accuracy: 0.8424 - val_loss: 2.3851 - val_accuracy: 0.5710
Epoch 4/5
5240/5240 [==============================] - 281s 54ms/step - loss: 0.4341 - accuracy: 0.8636 - val_loss: 1.9123 - val_accuracy: 0.6089
Epoch 5/5
5240/5240 [==============================] - 277s 53ms/step - loss: 0.3720 - accuracy: 0.8827 - val_loss: 2.1302 - val_accuracy: 0.5967


In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 136, 180)          1440000   
                                                                 
 flatten (Flatten)           (None, 24480)             0         
                                                                 
 dense (Dense)               (None, 1000)              24481000  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 800)               800800    
                                                                 
 dense_2 (Dense)             (None, 400)               320400    
                                                                 
 dropout_1 (Dropout)         (None, 400)              

In [43]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 136, 180)          1440000   
                                                                 
 gru_1 (GRU)                 (None, 400)               698400    
                                                                 
 flatten_4 (Flatten)         (None, 400)               0         
                                                                 
 dropout_6 (Dropout)         (None, 400)               0         
                                                                 
 dense_17 (Dense)            (None, 800)               320800    
                                                                 
 batch_normalization_6 (Batc  (None, 800)              3200      
 hNormalization)                                                 
                                                      

In [42]:
model = keras.models.load_model("model2.h5")

In [39]:
model.evaluate(padseq_test, y_test, batch_size=15)

 684/2184 [========>.....................] - ETA: 24s - loss: 1.1777 - accuracy: 0.6607

KeyboardInterrupt: 

In [16]:
mod = keras.Sequential([
    Input(shape=136),
    Embedding(numwords, 180)
])

2023-02-07 13:22:45.447842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 13:22:45.448422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-07 13:22:45.448691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-07 13:22:45.448847: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [34]:
model.evaluate(padseq_test, y_test)

1024/1024 [==============================] - 3s 3ms/step - loss: 1.9992 - accuracy: 0.5975


[1.9991627931594849, 0.5975390076637268]

In [23]:
padseq_train[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,   38, 1216,   97,
       1308, 2049,   97, 2049, 1216,   97,   10,  218,   18,  228,   20,
        462,   40, 8487, 1906], dtype=int32)

In [21]:
k2 = mod.predict(train_padseq[1])

5/5 [==============================] - 0s 935us/step


In [20]:
k.flatten()

array([ 0.03533294,  0.00897727, -0.02590462, ...,  0.01746919,
       -0.03165217,  0.02695059], dtype=float32)

In [22]:
k2.flatten()

array([ 0.03533294,  0.00897727, -0.02590462, ..., -0.04599941,
        0.02142418,  0.02448538], dtype=float32)

In [24]:
min([len(i) for i in train_seq])

0

In [30]:
model = keras.models.load_model("model.h5")

In [31]:
model.evaluate(padseq_test, y_test)

ValueError: in user code:

    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/parth/miniconda3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility

In [18]:
import keras.backend.tensorflow_backend
import tensorflow as tf
from tensorflow import keras

if keras.backend.tensorflow_backend._SESSION:   
   tf.reset_default_graph() 
   keras.backend.tensorflow_backend._SESSION.close()
   keras.backend.tensorflow_backend._SESSION = None

ModuleNotFoundError: No module named 'keras.backend.tensorflow_backend'; 'keras.backend' is not a package